<a href="https://colab.research.google.com/github/asmagh8/Recommendation_System_for_Purchase_Data.ipynb/blob/main/Recommendation_System_for_Purchase_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
%load_ext autoreload
%autoreload 2
import time

#from sklearn.model_selection import train_test_split
import sys
sys.path.append("..")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
!pip install turicreate


     |████████████████████████████████| 90.7MB 25kB/s 
     |████████████████████████████████| 17.3MB 140kB/s 
     |████████████████████████████████| 2.1MB 52.3MB/s 
     |████████████████████████████████| 327kB 56.3MB/s 
     |████████████████████████████████| 3.4MB 41.3MB/s 
     |████████████████████████████████| 23.8MB 100kB/s 
  Created wheel for resampy: filename=resampy-0.2.1-cp36-none-any.whl size=320847 sha256=8bdfba587ee0168998da500ce0db4308b9557bee403f89b744630ba4bb1c6751
  Stored in directory: /root/.cache/pip/wheels/ff/4f/ed/2e6c676c23efe5394bb40ade50662e90eb46e29b48324c5f9b
Successfully built resampy
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
ERROR: mxnet 1.1.0.post0 has requirement numpy<1.15.0,>=1.8.2, but you'll have numpy 1.16.4 which is incompatible.
ERROR: mxnet 1.1.0.post0 has requi

In [ ]:
import turicreate as tc

In [ ]:
data= pd.read_csv('/content/datamining.csv',encoding='ISO-8859-1')

In [ ]:

data['Description'] = data['Description'].str.strip()
data.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
data['InvoiceNo'] = data['InvoiceNo'].astype('str')
data = data[~data['InvoiceNo'].str.contains('C')]
data['CustomerID'] = data['CustomerID'].astype(str)
#data['CustomerID'] = data['CustomerID'].str.replace('.0', '')

In [ ]:
data = data.dropna(subset=['CustomerID'])
#data = data[pd.notnull(df['CustomerID'])]
#data.astype({'CustomerID': 'int32'}).dtypes
#data.astype({'CustomerID': 'category'}).dtypes'
#data['CustomerID']=data['CustomerID'].astype('category')
data.head(1000)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
1005,536520,21427,SKULLS STORAGE BOX SMALL,2,12/1/2010 12:43,2.10,14729.0,United Kingdom
1006,536520,21034,REX CASH+CARRY JUMBO SHOPPER,2,12/1/2010 12:43,0.95,14729.0,United Kingdom
1007,536520,21930,JUMBO STORAGE BAG SKULLS,1,12/1/2010 12:43,1.95,14729.0,United Kingdom
1008,536520,20772,GARDEN PATH JOURNAL,1,12/1/2010 12:43,2.55,14729.0,United Kingdom


In [ ]:
data = (data.groupby(['CustomerID', 'StockCode'])
    .agg({'StockCode': 'count'})
    .rename(columns={'StockCode': 'purchase_count'})
    .reset_index())

In [ ]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy
data_dummy = create_data_dummy(data)

In [ ]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='CustomerID', columns='StockCode')

In [ ]:
print(df_matrix)


StockCode   10002  10080  10120  ...  gift_0001_40  gift_0001_50    m
CustomerID                       ...                                 
12346.0       NaN    NaN    NaN  ...           NaN           NaN  NaN
12347.0       NaN    NaN    NaN  ...           NaN           NaN  NaN
12348.0       NaN    NaN    NaN  ...           NaN           NaN  NaN
12349.0       NaN    NaN    NaN  ...           NaN           NaN  NaN
12350.0       NaN    NaN    NaN  ...           NaN           NaN  NaN
...           ...    ...    ...  ...           ...           ...  ...
18281.0       NaN    NaN    NaN  ...           NaN           NaN  NaN
18282.0       NaN    NaN    NaN  ...           NaN           NaN  NaN
18283.0       NaN    NaN    NaN  ...           NaN           NaN  NaN
18287.0       NaN    NaN    NaN  ...           NaN           NaN  NaN
nan          24.0    3.0    NaN  ...           3.0           4.0  1.0

[4340 rows x 4059 columns]


In [ ]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())

In [ ]:
print(df_matrix_norm)

StockCode   10002  10080  10120  ...  gift_0001_40  gift_0001_50   m
CustomerID                       ...                                
12346.0       NaN    NaN    NaN  ...           NaN           NaN NaN
12347.0       NaN    NaN    NaN  ...           NaN           NaN NaN
12348.0       NaN    NaN    NaN  ...           NaN           NaN NaN
12349.0       NaN    NaN    NaN  ...           NaN           NaN NaN
12350.0       NaN    NaN    NaN  ...           NaN           NaN NaN
...           ...    ...    ...  ...           ...           ...  ..
18281.0       NaN    NaN    NaN  ...           NaN           NaN NaN
18282.0       NaN    NaN    NaN  ...           NaN           NaN NaN
18283.0       NaN    NaN    NaN  ...           NaN           NaN NaN
18287.0       NaN    NaN    NaN  ...           NaN           NaN NaN
nan           1.0    1.0    NaN  ...           NaN           NaN NaN

[4340 rows x 4059 columns]


In [ ]:
# create a table for input to the modeling
d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['CustomerID'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm

(268821, 3)


,CustomerID,StockCode,scaled_purchase_freq
86,12451.0,10002,0.000000
130,12510.0,10002,0.000000
189,12583.0,10002,0.000000
235,12637.0,10002,0.000000
266,12673.0,10002,0.000000
...,...,...,...
17585065,17444.0,POST,0.000000
17585319,17788.0,POST,0.000000
17585341,17828.0,POST,0.066667
17585424,17935.0,POST,0.000000


In [ ]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='CustomerID', columns='StockCode')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['CustomerID'], value_name='scaled_purchase_freq').dropna()

In [ ]:
def split_data(data):
    '''
    Splits dataset into training and test set.

    Args:
        data (pandas.DataFrame)

    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [ ]:
train_data, test_data = split_data(data)
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

In [ ]:
customers = data['CustomerID']
customers = set(customers)
print(customers)


{'16040.0', '13102.0', '12353.0', '17014.0', '15218.0', '15227.0', '17867.0', '15582.0', '15654.0', '16843.0', '17720.0', '12715.0', '13230.0', '17255.0', '14705.0', '15410.0', '12386.0', '14351.0', '14282.0', '15539.0', '17042.0', '14803.0', '14256.0', '14692.0', '18236.0', '15811.0', '13451.0', '17411.0', '15897.0', '13835.0', '16603.0', '16735.0', '13092.0', '13762.0', '14191.0', '15034.0', '12725.0', '12945.0', '18011.0', '18093.0', '12981.0', '13953.0', '14024.0', '16774.0', '12541.0', '15454.0', '14776.0', '14953.0', '12728.0', '15535.0', '16759.0', '15738.0', '13938.0', '13900.0', '17085.0', '15523.0', '12716.0', '14209.0', '17157.0', '13328.0', '16354.0', '13492.0', '14862.0', '17746.0', '16426.0', '17913.0', '14031.0', '13845.0', '13420.0', '16847.0', '14562.0', '17466.0', '13581.0', '18211.0', '15829.0', '17571.0', '13166.0', '14560.0', '18179.0', '14871.0', '13884.0', '14171.0', '12472.0', '16084.0', '14765.0', '17382.0', '15716.0', '17075.0', '13278.0', '15524.0', '16402.0'

In [ ]:
# constant variables to define field names include:
user_id = 'CustomerID'
item_id = 'StockCode'
users_to_recommend = list(customers)
n_rec = 10 # number of items to recommend
n_display = 30 # to display the first few rows in an output dataset

In [ ]:
def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data,
                                                    user_id=user_id,
                                                    item_id=item_id,
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data,
                                                    user_id=user_id,
                                                    item_id=item_id,
                                                    target=target,
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data,
                                                    user_id=user_id,
                                                    item_id=item_id,
                                                    target=target,
                                                    similarity_type='pearson')

    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [ ]:
name = 'popularity'
target = 'purchase_count'
popularity = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 216488 observations with 4316 users and 3967 items.

Data prepared in: 0.337698s

216488 observations to process; with 3967 unique items.

recommendations finished on 1000/4340 queries. users per second: 30195.1

recommendations finished on 2000/4340 queries. users per second: 31308.2

recommendations finished on 3000/4340 queries. users per second: 31614.2

recommendations finished on 4000/4340 queries. users per second: 30847.1

+------------+-----------+-------+------+
| CustomerID | StockCode | score | rank |
+------------+-----------+-------+------+
|  16040.0   |    DOT    | 693.0 |  1   |
|  16040.0   |   47566b  | 102.0 |  2   |
|  16040.0   |   84923   |  98.0 |  3   |
|  16040.0   |   72349b  |  82.0 |  4   |
|  16040.0   |   84997b  |  70.0 |  5   |
|  16040.0   |   84509a  |  68.0 |  6   |
|  16040.0   |   84970s  |  66.0 |  7   |
|  16040.0   |   84997c  |  66.0 |  8   |
|  16040.0   |   84997a  |  65.0 |  9   |
|  16040.0   |  15056bl  |  62.0 |  10  |
|  13102.0   |    DOT    | 693.0 |  1   |
|  13102.0   |   47566b  | 102.0 |  2   |
|  13102.0   |   84923   |  98.0 |  3   |
|  13102.0   |   72349b  |  82.0 |  4   |
|  13102.0   |   84997b  |  70.0 |  5   |
|  13102.0   |   84509a  |  68.0 |  6   |
|  13102.0   |   84970s  |  66.0 |  7   |
|  13102.0   |   84997c  |  66.0 |  8   |
|  13102.0   |   84997a  |  65.0 |  9   |
|  13102.0   |  15056bl  |  62.0 |  10  |
|  12353.0   |    DOT    | 693.0 |

In [ ]:
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 216488 observations with 4325 users and 3948 items.

Data prepared in: 0.339963s

...

recommendations finished on 1000/4340 queries. users per second: 32263.3

recommendations finished on 2000/4340 queries. users per second: 32921.3

recommendations finished on 3000/4340 queries. users per second: 33187.7

recommendations finished on 4000/4340 queries. users per second: 33284.5

+------------+-----------+-------+------+
| CustomerID | StockCode | score | rank |
+------------+-----------+-------+------+
|  16040.0   |   21109   |  1.0  |  1   |
|  16040.0   |   21504   |  1.0  |  2   |
|  16040.0   |   22111   |  1.0  |  3   |
|  16040.0   |   23349   |  1.0  |  4   |
|  16040.0   |   21609   |  1.0  |  5   |
|  16040.0   |   22699   |  1.0  |  6   |
|  16040.0   |   21484   |  1.0  |  7   |
|  16040.0   |   84380   |  1.0  |  8   |
|  16040.0   |   23636   |  1.0  |  9   |
|  16040.0   |   22992   |  1.0  |  10  |
|  13102.0   |   21109   |  1.0  |  1   |
|  13102.0   |   21504   |  1.0  |  2   |
|  13102.0   |   22111   |  1.0  |  3   |
|  13102.0   |   23349   |  1.0  |  4   |
|  13102.0   |   21609   |  1.0  |  5   |
|  13102.0   |   22699   |  1.0  |  6   |
|  13102.0   |   21484   |  1.0  |  7   |
|  13102.0   |   84380   |  1.0  |  8   |
|  13102.0   |   23636   |  1.0  |  9   |
|  13102.0   |   22992   |  1.0  |  10  |
|  12353.0   |   21109   |  1.0  |

In [ ]:
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 215056 observations with 4316 users and 3358 items.

Data prepared in: 0.367836s

215056 observations to process; with 3358 unique items.

recommendations finished on 1000/4340 queries. users per second: 33608.9

recommendations finished on 2000/4340 queries. users per second: 34961.4

recommendations finished on 3000/4340 queries. users per second: 35681.2

recommendations finished on 4000/4340 queries. users per second: 35770.2

+------------+-----------+-------+------+
| CustomerID | StockCode | score | rank |
+------------+-----------+-------+------+
|  16040.0   |   21816   |  1.0  |  1   |
|  16040.0   |   90199B  |  1.0  |  2   |
|  16040.0   |   90214W  |  1.0  |  3   |
|  16040.0   |   90179C  |  1.0  |  4   |
|  16040.0   |   35645   |  1.0  |  5   |
|  16040.0   |   90038A  |  1.0  |  6   |
|  16040.0   |   90214T  |  1.0  |  7   |
|  16040.0   |   90037B  |  1.0  |  8   |
|  16040.0   |   90033   |  1.0  |  9   |
|  16040.0   |   84743C  |  1.0  |  10  |
|  13102.0   |   21816   |  1.0  |  1   |
|  13102.0   |   90199B  |  1.0  |  2   |
|  13102.0   |   90214W  |  1.0  |  3   |
|  13102.0   |   90179C  |  1.0  |  4   |
|  13102.0   |   35645   |  1.0  |  5   |
|  13102.0   |   90038A  |  1.0  |  6   |
|  13102.0   |   90214T  |  1.0  |  7   |
|  13102.0   |   90037B  |  1.0  |  8   |
|  13102.0   |   90033   |  1.0  |  9   |
|  13102.0   |   84743C  |  1.0  |  10  |
|  12353.0   |   21816   |  1.0  |

In [ ]:
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 216488 observations with 4316 users and 3967 items.

Data prepared in: 0.345211s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 1000/4340 queries. users per second: 16852

recommendations finished on 2000/4340 queries. users per second: 16658.6

recommendations finished on 3000/4340 queries. users per second: 16695.9

recommendations finished on 4000/4340 queries. users per second: 16722.2

+------------+-----------+--------------------+------+
| CustomerID | StockCode |       score        | rank |
+------------+-----------+--------------------+------+
|  16040.0   |   23617   | 2.480882023656091  |  1   |
|  16040.0   |   72781   | 2.480882023656091  |  2   |
|  16040.0   |   90000C  | 2.480882023656091  |  3   |
|  16040.0   |   84964B  | 2.480882023656091  |  4   |
|  16040.0   |   85035c  | 2.480882023656091  |  5   |
|  16040.0   |  DCGS0004 | 2.4584846552028212 |  6   |
|  16040.0   |   21589   | 2.4584846552028212 |  7   |
|  16040.0   |   21765   | 2.4584846552028212 |  8   |
|  16040.0   |   85115B  | 2.4584846552028212 |  9   |
|  16040.0   |   47593b  | 2.4584846552028212 |  10  |
|  13102.0   |   72781   | 2.901506143030913  |  1   |
|  13102.0   |   84964B  | 2.901506143030913  |  2   |
|  13102.0   |   85035c  | 2.901506143030913  |  3   |
|  13102.0   |   84875A  | 2.901506143030913  |  4   |
|  13102.0   |   47593b  | 2.901506143030913  |  5   |
|  13102.0

In [ ]:
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 216488 observations with 4325 users and 3948 items.

Data prepared in: 0.361577s

...

recommendations finished on 1000/4340 queries. users per second: 17071.3

recommendations finished on 2000/4340 queries. users per second: 16424.4

recommendations finished on 3000/4340 queries. users per second: 15586.8

recommendations finished on 4000/4340 queries. users per second: 15330

+------------+-----------+----------------------+------+
| CustomerID | StockCode |        score         | rank |
+------------+-----------+----------------------+------+
|  16040.0   |   20728   | 0.07805505156517029  |  1   |
|  16040.0   |   22086   |  0.068877454996109   |  2   |
|  16040.0   |   22382   | 0.06272545576095581  |  3   |
|  16040.0   |   22383   |  0.0615358304977417  |  4   |
|  16040.0   |   22469   | 0.06021101713180542  |  5   |
|  16040.0   |   22470   | 0.059872502088546754 |  6   |
|  16040.0   |   21755   | 0.059864180088043215 |  7   |
|  16040.0   |   22384   |  0.0592099928855896  |  8   |
|  16040.0   |   23209   |  0.0579379403591156  |  9   |
|  16040.0   |   23203   | 0.056582560539245604 |  10  |
|  13102.0   |   23207   | 0.07108137597385634  |  1   |
|  13102.0   |   22382   | 0.05819862826257689  |  2   |
|  13102.0   |   20725   | 0.05441700546150534  |  3   |
|  13102.0   |   23203   | 0.05125447165252816  |  4   |
|  13102.0   |   22551   | 0.04

In [ ]:
name = 'cosine'
target = 'scaled_purchase_freq'
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 215056 observations with 4316 users and 3358 items.

Data prepared in: 0.383794s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 1000/4340 queries. users per second: 18286.2

recommendations finished on 2000/4340 queries. users per second: 17882

recommendations finished on 3000/4340 queries. users per second: 17251.8

recommendations finished on 4000/4340 queries. users per second: 16783.9

+------------+-----------+------------------------+------+
| CustomerID | StockCode |         score          | rank |
+------------+-----------+------------------------+------+
|  16040.0   |   22666   | 0.0006276117430792915  |  1   |
|  16040.0   |   21212   | 0.0006265984641181098  |  2   |
|  16040.0   |   22720   | 0.0005556000603569878  |  3   |
|  16040.0   |   85053   | 0.0003974040349324544  |  4   |
|  16040.0   |   22979   | 0.0003246890174018012  |  5   |
|  16040.0   |   22947   |  0.000323743290371365  |  6   |
|  16040.0   |   84596B  | 0.0003237366676330566  |  7   |
|  16040.0   |   22907   | 0.00032360818650987414 |  8   |
|  16040.0   |   15060B  | 0.00032330883873833553 |  9   |
|  16040.0   |   82581   | 0.0003231485684712728  |  10  |
|  13102.0   |   21912   | 0.0036621099168604072  |  1   |
|  13102.0   |   22378   |  0.00357061353596774   |  2   |
|  13102.0   |   23245   | 0.0034415017474781384  |  3   |
|  13102.0   |   35933   | 0.0034310086206956344  |  4  

In [ ]:
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 216488 observations with 4316 users and 3967 items.

Data prepared in: 0.378267s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 1000/4340 queries. users per second: 13960.4

recommendations finished on 2000/4340 queries. users per second: 14309.9

recommendations finished on 3000/4340 queries. users per second: 14633.9

recommendations finished on 4000/4340 queries. users per second: 14757.3

+------------+-----------+-------+------+
| CustomerID | StockCode | score | rank |
+------------+-----------+-------+------+
|  16040.0   |    DOT    | 693.0 |  1   |
|  16040.0   |   47566b  | 102.0 |  2   |
|  16040.0   |   84923   |  98.0 |  3   |
|  16040.0   |   72349b  |  82.0 |  4   |
|  16040.0   |   84997b  |  70.0 |  5   |
|  16040.0   |   84509a  |  68.0 |  6   |
|  16040.0   |   84970s  |  66.0 |  7   |
|  16040.0   |   84997c  |  66.0 |  8   |
|  16040.0   |   84997a  |  65.0 |  9   |
|  16040.0   |  15056bl  |  62.0 |  10  |
|  13102.0   |    DOT    | 693.0 |  1   |
|  13102.0   |   47566b  | 102.0 |  2   |
|  13102.0   |   84923   |  98.0 |  3   |
|  13102.0   |   72349b  |  82.0 |  4   |
|  13102.0   |   84997b  |  70.0 |  5   |
|  13102.0   |   84509a  |  68.0 |  6   |
|  13102.0   |   84970s  |  66.0 |  7   |
|  13102.0   |   84997c  |  66.0 |  8   |
|  13102.0   |   84997a  |  65.0 |  9   |
|  13102.0   |  15056bl  |  62.0 |  10  |
|  12353.0   |    DOT    | 693.0 |

In [ ]:
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 216488 observations with 4325 users and 3948 items.

Data prepared in: 0.349244s

...

recommendations finished on 1000/4340 queries. users per second: 17382.5

recommendations finished on 2000/4340 queries. users per second: 17765

recommendations finished on 3000/4340 queries. users per second: 17846.1

recommendations finished on 4000/4340 queries. users per second: 17888

+------------+-----------+-------+------+
| CustomerID | StockCode | score | rank |
+------------+-----------+-------+------+
|  16040.0   |   21109   |  0.0  |  1   |
|  16040.0   |   21504   |  0.0  |  2   |
|  16040.0   |   22111   |  0.0  |  3   |
|  16040.0   |   23349   |  0.0  |  4   |
|  16040.0   |   21609   |  0.0  |  5   |
|  16040.0   |   22699   |  0.0  |  6   |
|  16040.0   |   21484   |  0.0  |  7   |
|  16040.0   |   84380   |  0.0  |  8   |
|  16040.0   |   23636   |  0.0  |  9   |
|  16040.0   |   22992   |  0.0  |  10  |
|  13102.0   |   21109   |  0.0  |  1   |
|  13102.0   |   21504   |  0.0  |  2   |
|  13102.0   |   22111   |  0.0  |  3   |
|  13102.0   |   23349   |  0.0  |  4   |
|  13102.0   |   21609   |  0.0  |  5   |
|  13102.0   |   22699   |  0.0  |  6   |
|  13102.0   |   21484   |  0.0  |  7   |
|  13102.0   |   84380   |  0.0  |  8   |
|  13102.0   |   23636   |  0.0  |  9   |
|  13102.0   |   22992   |  0.0  |  10  |
|  12353.0   |   21109   |  0.0  |

In [ ]:
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 215056 observations with 4316 users and 3358 items.

Data prepared in: 0.357674s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 1000/4340 queries. users per second: 16165

recommendations finished on 2000/4340 queries. users per second: 15114.6

recommendations finished on 3000/4340 queries. users per second: 15116.9

recommendations finished on 4000/4340 queries. users per second: 15009.4

+------------+-----------+-------+------+
| CustomerID | StockCode | score | rank |
+------------+-----------+-------+------+
|  16040.0   |   21816   |  1.0  |  1   |
|  16040.0   |   90199B  |  1.0  |  2   |
|  16040.0   |   90214W  |  1.0  |  3   |
|  16040.0   |   90179C  |  1.0  |  4   |
|  16040.0   |   35645   |  1.0  |  5   |
|  16040.0   |   90038A  |  1.0  |  6   |
|  16040.0   |   90214T  |  1.0  |  7   |
|  16040.0   |   90037B  |  1.0  |  8   |
|  16040.0   |   90033   |  1.0  |  9   |
|  16040.0   |   84743C  |  1.0  |  10  |
|  13102.0   |   21816   |  1.0  |  1   |
|  13102.0   |   90199B  |  1.0  |  2   |
|  13102.0   |   90214W  |  1.0  |  3   |
|  13102.0   |   90179C  |  1.0  |  4   |
|  13102.0   |   35645   |  1.0  |  5   |
|  13102.0   |   90038A  |  1.0  |  6   |
|  13102.0   |   90214T  |  1.0  |  7   |
|  13102.0   |   90037B  |  1.0  |  8   |
|  13102.0   |   90033   |  1.0  |  9   |
|  13102.0   |   84743C  |  1.0  |  10  |
|  12353.0   |   21816   |  1.0  |

In [ ]:
models_w_counts = [popularity, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]
names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

In [ ]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Popularity Model on Purchase Counts


recommendations finished on 1000/4040 queries. users per second: 17900.9

recommendations finished on 2000/4040 queries. users per second: 16667.4

recommendations finished on 3000/4040 queries. users per second: 16480.6

recommendations finished on 4000/4040 queries. users per second: 16466.7

...

recommendations finished on 1000/4057 queries. users per second: 11155

recommendations finished on 2000/4057 queries. users per second: 10835.2

recommendations finished on 3000/4057 queries. users per second: 10917.6

recommendations finished on 4000/4057 queries. users per second: 10922.6


Precision and recall summary statistics by cutoff
+--------+------------------------+------------------------+
| cutoff |     mean_precision     |      mean_recall       |
+--------+------------------------+------------------------+
|   1    | 0.0004929751047572094  |  3.55718187835013e-05  |
|   2    | 0.0002464875523786047  | 3.557181878350128e-05  |
|   3    | 0.00041081258729767456 | 4.690569491643969e-05  |
|   4    | 0.0004313532166625589  | 7.807595026460884e-05  |
|   5    | 0.00039438008380576783 | 7.843526156545228e-05  |
|   6    | 0.0004518938460274425  | 0.00011069136516198523 |
|   7    | 0.00045776259727455134 |  0.000125178958820309  |
|   8    | 0.00040054227261523284 | 0.00012517895882030894 |
|   9    | 0.0003560375756579849  |  0.000125178958820309  |
|   10   | 0.00041902883904362863 | 0.0001654186920572806  |
+--------+------------------------+------------------------+
[10 rows x 3 columns]


Overall RMSE: 0.12791205120877477

Per User RMSE (best)
+------------+-